In [ ]:
import re
import copy
import torch
import json
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

In [2]:
with open("../data/mma_science_posts.json", "r", encoding="utf-8") as f:
    raw_posts = json.load(f)

In [ ]:
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        "\U0001f600-\U0001f64f"
        "\U0001f300-\U0001f5ff"
        "\U0001f680-\U0001f6ff"
        "\U0001f700-\U0001f77f"
        "\U0001f780-\U0001f7ff"
        "\U0001f800-\U0001f8ff" 
        "\U0001f900-\U0001f9ff" 
        "\U0001fa00-\U0001fa6f"
        "\U0001fa70-\U0001faff"
        "\U00002702-\U000027b0"
        "\U000024c2-\U0001f251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", text)

In [ ]:
dataset = []
for post in raw_posts:
    text = post.get("text", "").strip()
    if len(text) > 500:
        clean_text = remove_emojis(text)
        dataset.append(
            {"prompt": "напиши пост о mma в стиле mma science", "response": clean_text}
        )

with open("../data/mma_dataset.jsonl", "w", encoding="utf-8") as f:
    for item in dataset:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [2]:
dataset = load_dataset("json", data_files="../data/mma_dataset_tuned.jsonl", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
def remove_space(batch):
    batch['response'] = [row.lstrip() for row in batch['response']]
    return batch

In [13]:
dataset = dataset.map(remove_space, batched=True)

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

In [ ]:
model_name = "ai-forever/rugpt3medium_based_on_gpt2"

In [16]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [18]:
prompt_template = """Ниже приведена инструкция, описывающая задачу. Напиши ответ, который соответствующим образом завершает запрос. Инструкция: {instruction}\n Ответ:"""
answer_template = """{response}"""

In [19]:
def add_text(rec):
    instruction = rec["prompt"]
    response = rec["response"]
    if not instruction:
        raise ValueError(f"Expected an instruction in: {rec}")
    if not response:
        raise ValueError(f"Expected a response in: {rec}")
    rec["prompt"] = prompt_template.format(instruction=instruction)
    rec["answer"] = answer_template.format(response=response)
    rec["text"] = rec["prompt"] + rec["answer"]
    return rec

In [20]:
new_dataset = dataset.map(add_text)

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

In [21]:
def preprocess_batch(batch):
    model_inputs = tokenizer(
        batch["text"], max_length=512, truncation=True, padding="max_length"
    )
    model_inputs["labels"] = copy.deepcopy(model_inputs["input_ids"])
    return model_inputs

In [19]:
new_dataset[0]

{'prompt': 'Ниже приведена инструкция, описывающая задачу. Напиши ответ, который соответствующим образом завершает запрос. Инструкция: напиши пост о mma в стиле mma science\n Ответ:',
 'response': '\u200b\u200bПродолжение поста \n\nТопурия очень не любит, когда соперник режет углы. В таких случаях он проваливается и теряет энергию на выбрасывание тяжелых ударов. Волкановски и Холлоуэй понимали это и постоянно кружили вокруг грузина. Особенно хорошо это получалось у Макса. В первом раунде он даже перебил чемпа, не пропустив ничего серьезного. Илия предпочитает боковые и оверхенды, но для них нужна еще короче дистанция, чем для прямых. Он никак не мог подобраться к гавайцу и в конце концов поймал того именно прямым с быстрым сокращением дистанции. Поэтому Оливейра должен был весь лагерь тренировать сайд степы, пивоты и прочие смещения. Ему нужно двигаться преимущественно вправо, дальше от сильнейшей руки Топурии. Но одной только защитой бои не забираются. Поэтому бывшему чемпу нужно пост

In [22]:
tokenized = new_dataset.map(
    preprocess_batch,
    batched=True,
    remove_columns=["prompt", "response", "answer", "text"],
)

Map:   0%|          | 0/138 [00:00<?, ? examples/s]

In [ ]:
lora_config = LoraConfig(
    r=24,
    lora_alpha=24,
    target_modules=["c_attn", "c_proj"],
    lora_dropout=0.2,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, lora_config)

In [11]:
tokenized = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)

Map:   0%|          | 0/136 [00:00<?, ? examples/s]

In [24]:
args = TrainingArguments(
    output_dir="./mma_science_model",
    per_device_train_batch_size=2,
    # gradient_accumulation_steps=4,
    # warmup_steps=10,
    learning_rate=1e-3,
    num_train_epochs=50,
    logging_steps=50,
    # logging_dir="./logs",
    save_strategy="epoch",
    label_names=["labels"],
    fp16=True,
    remove_unused_columns=False,
    report_to="none",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
    processing_class=tokenizer,
)
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,4.921000
100,3.605400
150,3.311200
200,3.133200
250,2.870800
300,2.870200
350,2.693500
400,2.545000
450,2.390400
500,2.348400


TrainOutput(global_step=3450, training_loss=1.0626074036307958, metrics={'train_runtime': 575.5847, 'train_samples_per_second': 11.988, 'train_steps_per_second': 5.994, 'total_flos': 6545560987238400.0, 'train_loss': 1.0626074036307958, 'epoch': 50.0})

In [26]:
model.save_pretrained("./ru_gpt_tuned")
tokenizer.save_pretrained("./ru_gpt_tuned")

('./ru_gpt_tuned/tokenizer_config.json',
 './ru_gpt_tuned/special_tokens_map.json',
 './ru_gpt_tuned/vocab.json',
 './ru_gpt_tuned/merges.txt',
 './ru_gpt_tuned/added_tokens.json',
 './ru_gpt_tuned/tokenizer.json')